In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm # 폰트연결 시켜주는 모듈
font_name = fm.FontProperties(fname = "C:\\Windows\\Fonts\\malgun.ttf").get_name()
plt.rc("font", family = font_name)

# 마이너스 사인 해결
import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Flatten, Dense, Input, concatenate
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import SGD, Adam
import matplotlib .pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score

from datetime import datetime
import cv2
import zipfile
import shutil
import os
import glob

from tensorflow.keras.applications import Xception, ResNet50
from tensorflow.keras.applications import InceptionV3, MobileNet

In [3]:
import mglearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.datasets import load_iris, fetch_california_housing, load_breast_cancer
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report, roc_curve
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC, SVR
from sklearn.svm import LinearSVC, LinearSVR
from sklearn.metrics import mean_squared_error

from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.cluster import KMeans

from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

## 1번 농장 환경데이터 (시계열)

In [4]:
# def mk_df(path):
    
name = ["00_(환경제어기, 양액기)", "01_(함수율저울,열화상센서,작물환경센서)"]
file_path = "project/smart_tomato/01.데이터/1.Training/원천데이터/TS_Timeseries"
file_list = []
env_list = []
dfs = []
to_file = os.path.join(file_path, "tom1")
print(to_file)


for env in name:
    to_file_env = os.path.join(to_file, env)
    env_list.append(to_file_env)

for file in env_list:
    file_name = glob.glob(file+"/*csv")
    
    for names in file_name:
        file_list.append(names)
    
for i in range(len(file_list)):
    
    try:
        df = pd.read_csv(file_list[i], encoding="cp949")
        
    except UnicodeDecodeError as e :
        
        df = pd.read_csv(file_list[i], encoding="utf-8")
        
    dfs.append(df)

project/smart_tomato/01.데이터/1.Training/원천데이터/TS_Timeseries\tom1


In [5]:
len(dfs)

19

In [6]:
dfs[0]

,채널1 구역,날짜,시간,설정량1,설정량2,설정량3,잔여량1,잔여량2,잔여량3,누적량1,...,종료,설정EC(dS),현재EC(dS),설정PH(pH),현재PH(pH),일출시간,일몰시간,현재일사(W),누적일사(J),J/Day
0,0,2021-9-12,23:0:0,100,100,100,0,0,0,1599,...,15:36,2.8,3.5,5,4.3,6:13,18:46,0,2052,963
1,0,2021-9-12,23:2:0,100,100,100,0,0,0,1599,...,15:36,2.8,3.4,5,4.3,6:13,18:46,0,2052,963
2,0,2021-9-12,23:3:58,100,100,100,0,0,0,1599,...,15:36,2.8,3.5,5,4.3,6:13,18:46,0,2052,963
3,0,2021-9-12,23:6:2,100,100,100,0,0,0,1599,...,15:36,2.8,3.5,5,4.2,6:13,18:46,0,2052,963
4,0,2021-9-12,23:8:0,100,100,100,0,0,0,1599,...,15:36,2.8,3.5,5,4.2,6:13,18:46,0,2052,963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9898,0,2021-9-27,10:45:38,90,90,90,0,0,0,399,...,15:13,2.8,3.4,5,5.0,6:25,18:23,309,410,1618
9899,0,2021-9-27,10:47:37,90,90,90,0,0,0,399,...,15:13,2.8,3.4,5,5.0,6:25,18:23,324,414,1618
9900,0,2021-9-27,10:49:37,90,90,90,0,0,0,399,...,15:13,2.8,3.4,5,4.9,6:25,18:23,307,418,1618
9901,0,2021-9-27,10:51:37,90,90,90,0,0,0,399,...,15:13,2.8,3.4,5,4.9,6:25,18:23,999,424,1618


In [7]:
# 환경제어기 데이터와 함수율저율/열화상센서 데이터 나누기
dfs_0 = dfs[:-1] # 00_(환경제어기, 양액기)
dfs_1 = dfs[-1] # 01_(함수율저울,열화상센서,작물환경센서)

In [8]:
len(dfs_0)

18

In [33]:
column_list = []
for i in range(18):
    column_list.append(dfs_0[i].columns)

col_df = pd.DataFrame(column_list)
col_df
# 6번 df 1칸 밀림
# 16번 df 컬럼이 없어보임

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,채널1 구역,날짜,시간,설정량1,설정량2,설정량3,잔여량1,잔여량2,잔여량3,누적량1,...,설정EC(dS),현재EC(dS),설정PH(pH),현재PH(pH),일출시간,일몰시간,현재일사(W),누적일사(J),J/Day,None
1,채널1 구역,날짜,시간,설정량1,설정량2,설정량3,잔여량1,잔여량2,잔여량3,누적량1,...,설정EC(dS),현재EC(dS),설정PH(pH),현재PH(pH),일출시간,일몰시간,현재일사(W),누적일사(J),J/Day,None
2,채널1 구역,날짜,시간,설정량1,설정량2,설정량3,잔여량1,잔여량2,잔여량3,누적량1,...,설정EC(dS),현재EC(dS),설정PH(pH),현재PH(pH),일출시간,일몰시간,현재일사(W),누적일사(J),J/Day,None
3,채널1 구역,날짜,시간,설정량1,설정량2,설정량3,잔여량1,잔여량2,잔여량3,누적량1,...,설정EC(dS),현재EC(dS),설정PH(pH),현재PH(pH),일출시간,일몰시간,현재일사(W),누적일사(J),J/Day,None
4,채널1 구역,날짜,시간,설정량1,설정량2,설정량3,잔여량1,잔여량2,잔여량3,누적량1,...,설정EC(dS),현재EC(dS),설정PH(pH),현재PH(pH),일출시간,일몰시간,현재일사(W),누적일사(J),J/Day,None
5,,채널1 구역,날짜,시간,설정량1,설정량2,설정량3,잔여량1,잔여량2,잔여량3,...,종료,설정EC(dS),현재EC(dS),설정PH(pH),현재PH(pH),일출시간,일몰시간,현재일사(W),누적일사(J),J/Day
6,채널1 구역,날짜,시간,설정량1,설정량2,설정량3,잔여량1,잔여량2,잔여량3,누적량1,...,설정EC(dS),현재EC(dS),설정PH(pH),현재PH(pH),일출시간,일몰시간,현재일사(W),누적일사(J),J/Day,None
7,채널1 구역,날짜,시간,설정량1,설정량2,설정량3,잔여량1,잔여량2,잔여량3,누적량1,...,설정EC(dS),현재EC(dS),설정PH(pH),현재PH(pH),일출시간,일몰시간,현재일사(W),누적일사(J),J/Day,None
8,채널1 구역,날짜,시간,설정량1,설정량2,설정량3,잔여량1,잔여량2,잔여량3,누적량1,...,설정EC(dS),현재EC(dS),설정PH(pH),현재PH(pH),일출시간,일몰시간,현재일사(W),누적일사(J),J/Day,None
9,채널1 구역,날짜,시간,설정량1,설정량2,설정량3,잔여량1,잔여량2,잔여량3,누적량1,...,설정EC(dS),현재EC(dS),설정PH(pH),현재PH(pH),일출시간,일몰시간,현재일사(W),누적일사(J),J/Day,None


In [35]:
for i in range(18):
    print(col_df.loc[i].count())
# 예상되로 6번, 16번 컬럼 이상

23
23
23
23
23
24
23
23
23
23
23
23
23
23
23
9
23
23


In [48]:
# 23개 컬럼 맞춰주기
dfs_0[5] = dfs_0[5][dfs_0[5].columns[1:]]
dfs_0[5]

,채널1 구역,날짜,시간,설정량1,설정량2,설정량3,잔여량1,잔여량2,잔여량3,누적량1,...,종료,설정EC(dS),현재EC(dS),설정PH(pH),현재PH(pH),일출시간,일몰시간,현재일사(W),누적일사(J),J/Day
0,0,2021-10-17,23:1:7,90,90,90,0,0,0,1399,...,14:44,2.8,3,5,4.2,6:43,17:54,0,0,558
1,0,2021-10-17,23:3:6,90,90,90,0,0,0,1399,...,14:44,2.8,3,5,4.1,6:43,17:54,0,0,558
2,0,2021-10-17,23:5:5,90,90,90,0,0,0,1399,...,14:44,2.8,3,5,4.2,6:43,17:54,0,0,558
3,0,2021-10-17,23:7:9,90,90,90,0,0,0,1399,...,14:44,2.8,3,5,4.2,6:43,17:54,0,0,558
4,0,2021-10-17,23:9:9,90,90,90,0,0,0,1399,...,14:44,2.8,3,5,4.2,6:43,17:54,0,0,558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10353,0,2021-11-1,10:5:27,120,120,120,0,0,0,240,...,14:25,2.8,3.3,4.5,5.1,6:58,17:35,432,226,0
10354,0,2021-11-1,10:7:30,120,120,120,0,0,0,240,...,14:25,2.8,3.3,4.5,5.1,6:58,17:35,406,232,0
10355,0,2021-11-1,10:9:33,120,120,120,0,0,0,240,...,14:25,2.8,3.3,4.5,5.2,6:58,17:35,444,237,0
10356,1,2021-11-1,10:11:28,120,120,120,98,120,120,261,...,14:25,2.8,2.7,4.5,5.3,6:58,17:35,511,243,0


In [49]:
# 16번 df의 정보 불분명 (20220103 ~ 20220117 데이터 제외처리)
dfs_0[15]

,1437,17.49,15.944,18.434,0,-0.004,2.954,7.768,220102-2357
0,1438,17.490,15.942,18.434,0,-0.005,2.953,7.769,220102-2358
1,1439,17.488,15.942,18.432,0,-0.005,2.953,7.765,220102-2359
2,1440,17.486,15.940,18.430,0,-0.005,2.953,7.766,220103-0000
3,1441,17.486,15.938,18.428,0,-0.005,2.953,7.766,220103-0001
4,1442,17.484,15.936,18.428,0,-0.005,2.952,7.765,220103-0002
...,...,...,...,...,...,...,...,...,...
21158,22596,18.508,17.906,19.532,0,4.793,5.785,8.038,220117-1635
21159,22597,18.508,17.906,19.530,0,4.793,5.783,8.038,220117-1636
21160,22598,18.506,17.904,19.528,0,4.793,5.783,8.038,220117-1637
21161,22599,18.506,17.902,19.528,0,4.793,5.783,8.038,220117-1638


In [50]:
con_df1 = pd.concat([dfs_0[0], dfs_0[1]], axis = 0)
con_df2 = pd.concat([con_df1, dfs_0[2]], axis = 0)
con_df3 = pd.concat([con_df2, dfs_0[3]], axis = 0)
con_df4 = pd.concat([con_df3, dfs_0[4]], axis = 0)
con_df5 = pd.concat([con_df4, dfs_0[5]], axis = 0)
con_df6 = pd.concat([con_df5, dfs_0[6]], axis = 0)
con_df7 = pd.concat([con_df6, dfs_0[7]], axis = 0)
con_df8 = pd.concat([con_df7, dfs_0[8]], axis = 0)
con_df9 = pd.concat([con_df8, dfs_0[9]], axis = 0)
con_df10 = pd.concat([con_df9, dfs_0[10]], axis = 0)
con_df11 = pd.concat([con_df10, dfs_0[11]], axis = 0)
con_df12 = pd.concat([con_df11, dfs_0[12]], axis = 0)
con_df13 = pd.concat([con_df12, dfs_0[13]], axis = 0)
con_df14 = pd.concat([con_df13, dfs_0[14]], axis = 0)
con_df15 = pd.concat([con_df14, dfs_0[16]], axis = 0)
con_df16 = pd.concat([con_df15, dfs_0[17]], axis = 0)

In [52]:
df = con_df16
df

,채널1 구역,날짜,시간,설정량1,설정량2,설정량3,잔여량1,잔여량2,잔여량3,누적량1,...,종료,설정EC(dS),현재EC(dS),설정PH(pH),현재PH(pH),일출시간,일몰시간,현재일사(W),누적일사(J),J/Day
0,0,2021-9-12,23:0:0,100,100,100,0,0,0,1599,...,15:36,2.8,3.5,5,4.3,6:13,18:46,0,2052,963
1,0,2021-9-12,23:2:0,100,100,100,0,0,0,1599,...,15:36,2.8,3.4,5,4.3,6:13,18:46,0,2052,963
2,0,2021-9-12,23:3:58,100,100,100,0,0,0,1599,...,15:36,2.8,3.5,5,4.3,6:13,18:46,0,2052,963
3,0,2021-9-12,23:6:2,100,100,100,0,0,0,1599,...,15:36,2.8,3.5,5,4.2,6:13,18:46,0,2052,963
4,0,2021-9-12,23:8:0,100,100,100,0,0,0,1599,...,15:36,2.8,3.5,5,4.2,6:13,18:46,0,2052,963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15662,0,2022-2-7,17:41:33,80,80,80,0,0,0,629,...,14:13,2.8,2.7,4.8,4.5,7:33,18:3,5,793,760
15663,0,2022-2-7,17:43:33,80,80,80,0,0,0,629,...,14:13,2.8,2.8,4.8,4.5,7:33,18:3,5,793,760
15664,0,2022-2-7,17:45:33,80,80,80,0,0,0,629,...,14:13,2.8,2.7,4.8,4.5,7:33,18:3,2,793,760
15665,0,2022-2-7,17:47:34,80,80,80,0,0,0,629,...,14:13,2.8,2.7,4.8,4.5,7:33,18:3,0,793,760


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178164 entries, 0 to 15666
Data columns (total 23 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   채널1 구역      178164 non-null  object
 1    날짜         178164 non-null  object
 2    시간         178164 non-null  object
 3    설정량1       178164 non-null  object
 4    설정량2       178164 non-null  object
 5    설정량3       178164 non-null  object
 6    잔여량1       178164 non-null  object
 7    잔여량2       178164 non-null  object
 8    잔여량3       178164 non-null  object
 9    누적량1       178164 non-null  object
 10   누적량2       178164 non-null  object
 11   누적량3       178164 non-null  object
 12   시작         178164 non-null  object
 13   종료         178164 non-null  object
 14   설정EC(dS)   178164 non-null  object
 15   현재EC(dS)   178164 non-null  object
 16   설정PH(pH)   178164 non-null  object
 17   현재PH(pH)   178164 non-null  object
 18   일출시간       178164 non-null  object
 19   일몰시간       178164 non-n

In [69]:
# 띄어쓰기 난무되어있는 컬럼명 수정
cols = ['채널1 구역', '날짜', '시간', '설정량1', '설정량2', '설정량3', '잔여량1',
       '잔여량2', '잔여량3', '누적량1', '누적량2', '누적량3', '시작', '종료',
       '설정EC(dS)', '현재EC(dS)', '설정PH(pH)', '현재PH(pH)', '일출시간',
       '일몰시간', '현재일사(W)', '누적일사(J)', 'J/Day']
df.columns = cols

In [70]:
df.columns

Index(['채널1 구역', '날짜', '시간', '설정량1', '설정량2', '설정량3', '잔여량1', '잔여량2', '잔여량3',
       '누적량1', '누적량2', '누적량3', '시작', '종료', '설정EC(dS)', '현재EC(dS)', '설정PH(pH)',
       '현재PH(pH)', '일출시간', '일몰시간', '현재일사(W)', '누적일사(J)', 'J/Day'],
      dtype='object')

In [77]:
df.to_csv("project/farm1_env_data.csv", encoding = "utf-8")